# Visualize data and create dashboard

<hr>
Author: Jesse Huang 

Date created: July 20, 2023 

Date last updated: July 21, 2023

<hr>

Using rocket launch dataset obtained from SpaceX API, we will explore and better understand the available measures.

Import required libraries

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

In [2]:
# Read Launch site data
spacex_dt = pd.read_csv("data/spacex_launch_geo.csv")
spacex_dt = spacex_dt[['Launch Site', 'Lat', 'Long', 'class']]
spacex_dt.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [3]:
launch_coor = spacex_dt.drop_duplicates('Launch Site')
launch_coor

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
26,VAFB SLC-4E,34.632834,-120.610745,0
36,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1


## [1] Plot USA Map and mark launch sites by coordinate

In [4]:
site_map = folium.Map(location=[29.5597, -95.0831], zoom_start=4.2) #<create map object
for name, coor in [[x[0],[x[1], x[2]]] for x in launch_coor.values.tolist()]:
    folium.Circle(coor, radius=1000, color='#d35400', fill=True).add_to(site_map)
    folium.Marker(coor, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{name}</b></div>')).add_to(site_map)
site_map

Most launches are on the east-coast of Florida with only VAFB-SLC4E on the coast of California.

## [2] Cluster points by Launch Success

In [5]:
marker_allLaunch = MarkerCluster()
site_map.add_child(marker_allLaunch)
for index, record in spacex_dt.iterrows():
    if record['class']==1: markerColor = "green"
    else: markerColor = "red"
    marker_allLaunch.add_child(folium.Marker([record['Lat'],record['Long']],
                                             icon=folium.Icon(color='white', icon_color=markerColor)))
site_map

Most successful launches came from Florida (KSC LC-39A).

## [3] Find distance of Launchsite to Coast

Use function to calculate distance between to points

In [6]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Get coordinate of location by hovering mouse over map.

In [7]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Find the closest Coast, Highway, City, and Railway

In [8]:
# For KSC LC-39A: 28.573255, -80.646895
# Closest Coast: 28.57156, -80.60635
# Closest Highway: 28.57344, -80.65544
# Closest City: 28.61286, -80.81097
# Closets Railway: 28.57367, -80.65422
d1a = calculate_distance(28.573255, -80.646895, 28.57156, -80.60635)
d1b = calculate_distance(28.573255, -80.646895, 28.57344, -80.65544)
d1c = calculate_distance(28.573255, -80.646895, 28.61286, -80.81097)
d1d = calculate_distance(28.573255, -80.646895, 28.57367, -80.65422)


# For VAFB SLC-4E 34.632834, -120.610745
# Closest Coast: 34.63557, -120.62572
# Closest Highway: 34.63653, -120.6222
# Closest City: 34.639, -120.45874
# Closets Railway: 34.63702, -120.62336
d2a = calculate_distance(34.632834, -120.610745, 34.63557, -120.62572)
d2b = calculate_distance(34.632834, -120.610745, 34.63653, -120.6222)
d2c = calculate_distance(34.632834, -120.610745, 34.63745, -120.45874)
d2d = calculate_distance(34.632834, -120.610745, 34.63702, -120.62336)

In [9]:
folium.Marker([28.57156, -80.60635], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d1a,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[28.573255, -80.646895],[28.57156, -80.60635]], weight=1).add_to(site_map)

folium.Marker([28.57344, -80.65544], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d1b,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[28.573255, -80.646895],[28.57344, -80.65544]], weight=1).add_to(site_map)

folium.Marker([28.61286, -80.81097], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d1c,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[28.573255, -80.646895],[28.61286, -80.81097]], weight=1).add_to(site_map)

folium.Marker([28.57367, -80.65422], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d1d,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[28.573255, -80.646895],[28.57367, -80.65422]], weight=1).add_to(site_map)


folium.Marker([34.63557, -120.62572], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d2a,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[34.632834, -120.610745],[34.63557, -120.62572]], weight=1).add_to(site_map)

folium.Marker([34.63653, -120.6222], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d2b,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[34.632834, -120.610745],[34.63653, -120.6222]], weight=1).add_to(site_map)

folium.Marker([34.63745, -120.45874], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d2c,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[34.632834, -120.610745],[34.63745, -120.45874]], weight=1).add_to(site_map)

folium.Marker([34.63702, -120.62336], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html=f'<div style="font-size: 22; color:#d35400;"><b>{round(d2d,2)} KM</b></div>')).add_to(site_map)
folium.PolyLine([[34.632834, -120.610745],[34.63702, -120.62336]], weight=1).add_to(site_map)


site_map

As expected, space rocket launch sites are chosen to be far from populated cities while being close to coast, railway, and highway for easier transportation of material.